
# Ruta óptima con plano real (CSV) **+ tiempo de pick** integrado (TSP y VRP)

Este notebook extiende la versión anterior e **integra el tiempo de pick** (`pick_time_seconds`) en el **costo total** de la ruta para:
- 1 picker (TSP con Nearest Neighbor + 2-opt)
- VRP simple (Clarke–Wright) con capacidad por número de ítems.

**Archivos esperados** en `data/` (ya hay ejemplos listos):
- `Nodos_de_tienda_tienda_nodes.csv` — columnas: `node_id, x, y, kind` (usa `kind=depot` para inicio, `kind=packout` para destino)
- `Aristas_pasillos_tienda_edges.csv` — columnas: `u, v, weight, one_way` (1 = dirigido u→v; 0 = se agregan u→v y v→u)
- `Pedido_Items_pedido_items.csv` — columnas: `item_id, node_id, volume, pick_time_seconds`


In [ ]:

# === Imports ===
import math, itertools
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

NODES_CSV = 'data/Nodos_de_tienda_tienda_nodes.csv'
EDGES_CSV = 'data/Aristas_pasillos_tienda_edges.csv'
ITEMS_CSV = 'data/Pedido_Items_pedido_items.csv'



## 1) Cargar CSVs y construir grafo
Se **agregan** ítems por **nodo** para manejar múltiples SKUs en la misma ubicación:
- `volume_by_node` = suma de `volume`
- `pick_time_by_node` = suma de `pick_time_seconds`


In [ ]:

nodes = pd.read_csv(NODES_CSV)
edges = pd.read_csv(EDGES_CSV)
items = pd.read_csv(ITEMS_CSV)

# Agregación por nodo (múltiples SKUs en misma ubicación)
node_agg = items.groupby('node_id').agg(
    volume=('volume','sum'),
    pick_time_seconds=('pick_time_seconds','sum')
).reset_index()

order_nodes = node_agg['node_id'].tolist()
volume_by_node = dict(zip(node_agg['node_id'], node_agg['volume']))
pick_time_by_node = dict(zip(node_agg['node_id'], node_agg['pick_time_seconds']))

# Grafo dirigido
G = nx.DiGraph()
positions = {}
for _, r in nodes.iterrows():
    nid = r['node_id']
    positions[nid] = (float(r['x']), -float(r['y']))  # y invertida para dibujar hacia abajo
    G.add_node(nid, kind=r.get('kind','unknown'))

for _, r in edges.iterrows():
    u, v, w, ow = r['u'], r['v'], float(r['weight']), int(r.get('one_way',0))
    G.add_edge(u, v, weight=w)
    if ow == 0:
        G.add_edge(v, u, weight=w)

depot_start = nodes.loc[nodes['kind']=='depot', 'node_id'].iloc[0] if (nodes['kind']=='depot').any() else nodes['node_id'].iloc[0]
packout_candidates = nodes.loc[nodes['kind']=='packout', 'node_id']
depot_end = packout_candidates.iloc[0] if len(packout_candidates) else depot_start

print("Depot inicio:", depot_start, "| Depot fin:", depot_end)
print("Nodos a visitar (agregados por ubicación):", order_nodes)
print("Total pick_time (s):", sum(pick_time_by_node.values()))



## 2) Matriz de distancias entre puntos relevantes (solo **caminar**)
- Distancias mínimas en el grafo (no incluyen pick).
- El **costo total** se calculará como **caminar + pick** más adelante.


In [ ]:

def shortest_time_length(G, a, b):
    return nx.shortest_path_length(G, source=a, target=b, weight='weight')

def shortest_path_nodes(G, a, b):
    return nx.shortest_path(G, source=a, target=b, weight='weight')

P = [depot_start] + order_nodes + [depot_end]
D = {a:{} for a in P}
for a in P:
    for b in P:
        if a == b: continue
        D[a][b] = shortest_time_length(G, a, b)

print("Ejemplo D[depot_start][primer nodo]:", D[depot_start][order_nodes[0]])



## 3) TSP 1-picker con **pick time** integrado
- La función de costo suma **caminar** + **pick_time** por cada nodo visitado.


In [ ]:

def travel_cost(order, D, start, end):
    if not order:
        return 0
    cost = D[start][order[0]]
    for a,b in zip(order, order[1:]):
        cost += D[a][b]
    cost += D[order[-1]][end]
    return cost

def total_cost_with_pick(order, D, start, end, pick_time_by_node):
    return travel_cost(order, D, start, end) + sum(pick_time_by_node.get(n,0) for n in order)

def nearest_neighbor(order_nodes, D, start, end):
    # Selección por distancia de caminar (pick no cambia la elección relativa)
    unvisited = set(order_nodes)
    path = []
    curr = start
    while unvisited:
        nxt = min(unvisited, key=lambda x: D[curr][x])
        path.append(nxt)
        unvisited.remove(nxt)
        curr = nxt
    return path

def two_opt(path, D, start, end, pick_time_by_node):
    def total(p): return total_cost_with_pick(p, D, start, end, pick_time_by_node)
    best = path[:]
    best_cost = total(best)
    improved = True
    while improved:
        improved = False
        for i in range(1, len(best)-1):
            for k in range(i+1, len(best)):
                cand = best[:i] + best[i:k][::-1] + best[k:]
                c = total(cand)
                if c < best_cost:
                    best, best_cost = cand, c
                    improved = True
                    break
            if improved: break
    return best, best_cost

nn_path = nearest_neighbor(order_nodes, D, depot_start, depot_end)
opt_path, opt_total = two_opt(nn_path, D, depot_start, depot_end, pick_time_by_node)

t_travel = travel_cost(opt_path, D, depot_start, depot_end)
t_pick = sum(pick_time_by_node.get(n,0) for n in opt_path)
print("Ruta 1-picker:", opt_path)
print("  - Caminar:", t_travel)
print("  - Pick   :", t_pick)
print("  = Total  :", opt_total)



## 4) VRP simple (Clarke–Wright) **con pick integrado en el costo final**
- Capacidad por número de ítems (suma de `volume_by_node`).
- Los **ahorros** siguen usando **caminar** (estándar).
- El costo de cada ruta reporta: **caminar**, **pick** y **total**.


In [ ]:

depot_vrp = depot_start
num_vehicles = 2
capacity_per_vehicle = 3  # items (volumen) por vehículo

# Rutas iniciales: cada nodo en su propia ruta
routes = {n: [n] for n in order_nodes}
route_load = {n: volume_by_node.get(n,1) for n in order_nodes}

# Ahorros de Clarke–Wright (solo caminar)
savings = []
for i in order_nodes:
    for j in order_nodes:
        if i == j: continue
        s = D[depot_vrp][i] + D[j][depot_vrp] - D[i][j]
        savings.append((s, i, j))
savings.sort(reverse=True, key=lambda x: x[0])

def find_route(routes_dict, customer):
    for key, r in routes_dict.items():
        if customer in r:
            return key
    return None

for s, i, j in savings:
    ri = find_route(routes, i)
    rj = find_route(routes, j)
    if ri is None or rj is None or ri == rj:
        continue
    if routes[ri][-1] != i: continue   # i al final
    if routes[rj][0]  != j: continue   # j al inicio
    if route_load[ri] + route_load[rj] <= capacity_per_vehicle:
        routes[ri] = routes[ri] + routes[rj]
        route_load[ri] += route_load[rj]
        del routes[rj]

# Reducir a num_vehicles si es posible
while len(routes) > num_vehicles:
    keys = list(routes.keys())
    merged = False
    for a in keys:
        if merged: break
        for b in keys:
            if a == b or a not in routes or b not in routes: continue
            if route_load[a] + route_load[b] <= capacity_per_vehicle:
                routes[a] = routes[a] + routes[b]
                route_load[a] += route_load[b]
                del routes[b]
                merged = True
                break
    if not merged:
        break

# Costos por ruta (ida y vuelta al depósito) con pick integrado
vrp_routes = []
for key, r in routes.items():
    vrp_routes.append(r)

def roundtrip_travel_cost(order):
    if not order: return 0
    c = D[depot_vrp][order[0]]
    for a,b in zip(order, order[1:]):
        c += D[a][b]
    c += D[order[-1]][depot_vrp]
    return c

print("Rutas VRP (max", num_vehicles, "vehículos, cap", capacity_per_vehicle, "):")
for idx, r in enumerate(vrp_routes, 1):
    walk = roundtrip_travel_cost(r)
    pick = sum(pick_time_by_node.get(n,0) for n in r)
    total = walk + pick
    print(f"  Vehículo {idx}: {r}")
    print(f"    - Caminar: {walk}")
    print(f"    - Pick   : {pick}")
    print(f"    = Total  : {total}")



## 5) Visualización
Se dibuja el grafo y se resaltan los tramos de ruta (no distingue colores manualmente).


In [ ]:

def edges_in_shortest(G, a, b):
    p = shortest_path_nodes(G, a, b)
    return list(zip(p[:-1], p[1:]))

def draw_route(G, positions, seq_edges, title):
    plt.figure(figsize=(8,6))
    nx.draw_networkx_nodes(G, positions, node_size=400)
    nx.draw_networkx_edges(G, positions, arrows=True)
    nx.draw_networkx_labels(G, positions, font_size=8)
    nx.draw_networkx_edges(G, positions, edgelist=seq_edges, width=3, arrows=True)
    plt.axis("off")
    plt.title(title)
    plt.show()

# 1-picker
seq = [depot_start] + opt_path + [depot_end]
e1 = []
for a,b in zip(seq, seq[1:]):
    e1.extend(edges_in_shortest(G, a, b))
draw_route(G, positions, e1, "Ruta 1-picker (con pick en costo)")

# VRP: una figura por ruta
for idx, r in enumerate(vrp_routes, 1):
    seq = [depot_vrp] + r + [depot_vrp]
    egs = []
    for a,b in zip(seq, seq[1:]):
        egs.extend(edges_in_shortest(G, a, b))
    plt.figure(figsize=(8,6))
    nx.draw_networkx_nodes(G, positions, node_size=400)
    nx.draw_networkx_edges(G, positions, arrows=True)
    nx.draw_networkx_labels(G, positions, font_size=8)
    nx.draw_networkx_edges(G, positions, edgelist=egs, width=2+idx, arrows=True)
    plt.axis("off")
    plt.title(f"VRP Ruta vehículo {idx} (con pick en costo)")
    plt.show()



---

### Notas
- El **tiempo de pick** se suma **una sola vez por nodo visitado** y se **agrega al caminar** en el costo total.
- Si deseas imponer un **SLA de duración máxima por ruta** (p. ej., 900 s), puedes verificar `walk + pick <= SLA` y evitar fusiones que lo excedan.
- Para múltiples SKUs en el mismo nodo, el tiempo de pick es la **suma** de sus `pick_time_seconds`.
